In [1]:
import sklearn as sklearn
import pandas as pd
import numpy as np
from time import time
import seaborn as sns

In [2]:
pip install jupyterlab_github

Note: you may need to restart the kernel to use updated packages.
